In [2]:
import pytz

In [10]:
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
import seaborn as sns

In [5]:
from dateutil.parser import parse

In [6]:
from datetime import datetime, timedelta

In [7]:
from collections import defaultdict

In [8]:
from matplotlib.ticker import MaxNLocator

In [271]:
pd.set_option('display.max_colwidth', None)

In [17]:
df = pd.read_csv("/shared/3/projects/benlitterer/podcastData/processed/beforeFloydMonth/beforeFMonth.tsv", lineterminator='\n', low_memory=False)

In [19]:
df.columns

Index(['Unnamed: 0', 'rssUrl', 'epTitle', 'epDescription', 'duration',
       'pubDate', 'copyright', 'itunes:type', 'itunes:complete', 'guid',
       'itunes:explicit', 'enclosure', 'itunes:image', 'transDict', 'id',
       'title', 'lastUpdate', 'link', 'lastHttpStatus', 'dead', 'contentType',
       'itunesId', 'originalUrl', 'itunesAuthor', 'itunesOwnerName',
       'explicit', 'imageUrl', 'itunesType', 'generator', 'newestItemPubdate',
       'language', 'oldestItemPubdate', 'episodeCount', 'popularityScore',
       'priority', 'createdOn', 'updateFrequency', 'chash', 'host',
       'newestEnclosureUrl', 'podcastGuid', 'podDescription', 'category1',
       'category2', 'category3', 'category4', 'category5', 'category6',
       'category7', 'category8', 'category9', 'category10',
       'newestEnclosureDuration', 'oldestItemDatetime', 'cleanDates',
       'potentialOutPath', 'cleanDatesLoc'],
      dtype='object')

In [13]:
df = pd.read_csv("/shared/3/projects/benlitterer/podcastData/processed/allEpisodes.csv", lineterminator='\n', low_memory=False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [400]:
df.columns

Index(['Unnamed: 0', 'queryName', 'title', 'url', 'originalUrl', 'description',
       'author', 'language', 'categories', 'explicit_x', 'episodeCount',
       'epTitle', 'epDescription', 'duration', 'pubDate', 'copyright', 'type',
       'complete', 'guid', 'explicit_y', 'mp3Url', 'image', 'transDict',
       'date_formats_with_offset', 'date_formats_without_offset',
       'date_formats_without_PDT', 'date_format_same',
       'date_formate_same_final', 'date_format_same_final'],
      dtype='object')

In [267]:
# print(df[df['title'] == '上野誠の万葉歌ごよみ']['language'])
# print(df[df['title'] == '2017-07-31 21:59:35.00']['ex'])
# df['language'].value_counts()

In [287]:
# df.columns
# df.head(1)

In [28]:
# clean the explicit_y column. It contains a lot of inconsistent and confusing stuff.
# Will use explicit_x column for analysis below. Still helpful to clean explicit_y 
mapping = {
    'No': False,
    'no': False,
    'false': False,
    'clean': False,
    'yes': True,
    'true': True,
    'Yes': True,
    'NO': False,
    'False': False,
    'Clean': False,
    '    No': False,
    'N': False,
    'explicit': True,
    'YES': True,
    'Explicit': True,
    'off': False,
    'FALSE': False,
    'falses': False
}

# Apply the mapping to the explicit_y column
df['explicit_y'] = df['explicit_y'].map(mapping)

In [29]:
# -------- Clean date format: ----------

# Date formats:
date_formats_with_offset = "%a, %d %b %Y %H:%M:%S %z"
date_formats_without_offset = "%a, %d %b %Y %H:%M:%S %Z"
date_format_without_tzone = '%Y-%m-%d %H:%M:%S'

# clean date using only one format, leaving NaTs for unmatched format 
df['date_formats_with_offset'] = pd.to_datetime(df['pubDate'], errors='coerce', format=date_formats_with_offset, utc=True).dt.tz_localize(None)
df["date_formats_without_offset"] = pd.to_datetime(df['pubDate'], errors='coerce', format=date_formats_without_offset, utc=True).dt.tz_localize(None)
df["date_formats_without_PDT"] = pd.to_datetime(df['pubDate'].replace({'PDT':'-07:00','PST':'-08:00'}, regex=True), errors='coerce', utc=True).dt.tz_localize(None)


# Merge 3 columns  
df["date_format_same"] = df['date_formats_with_offset'].fillna(df["date_formats_without_offset"])
df['date_format_same_final'] = df["date_formats_without_PDT"].fillna(df["date_format_same"])

# -------- date format cleaned ----------



# # checks the amount of NaT
# count_of_NaT = df['date_format_same_final'].isna().sum()
# print("Amount of NaT values:",count_of_NaT)
# # the indices of NaT
# indices_of_NaT = df.index[df['date_format_same_final'].isna()].tolist()
# # print("Indices of NaT values:", indices_of_NaT)

In [ ]:
# checks the amount of NaT
print(df.shape)
count_of_NaT = df['date_format_same_final'].isna().sum()
print("Amount of NaT values:",count_of_NaT)
# the indices of NaT
indices_of_NaT = df.index[df['date_format_same_final'].isna()].tolist()
# print("Indices of NaT values:", indices_of_NaT)

In [30]:
# Filter out inconsistent podcasts
Df = pd.DataFrame()
Df['podcast title'] = df['title']
Df['publish date'] = df['date_format_same_final']
Df['category'] = df['categories']
Df['language'] = df['language']
Df['explicit'] = df['explicit_x']

Df = Df.dropna(subset=['publish date'])

grouped = Df.groupby(['podcast title', 'category', 'language', 'explicit'])['publish date'].agg(['min', 'max', 'median'])
grouped['time span'] = grouped['max'] - grouped['min']
grouped['median - min'] = abs(grouped['median'] - grouped['min'])
grouped['max - median'] = abs(grouped['max'] - grouped['median'])

# Set it to 5500D because the first 
consistent_podcasts = grouped[(grouped['median - min'] < pd.Timedelta('7000D')) & 
                             (grouped['max - median'] < pd.Timedelta('7000D'))]

consistent_sorted_podcasts = consistent_podcasts.sort_values('time span', ascending=False)
consistent_sorted_podcasts = consistent_sorted_podcasts.reset_index()

# Generate a CSV file that only has consistent podcasts:
# consistent_sorted_podcasts.to_csv('consistent_podcasts.csv')

In [46]:
# ECDF plot (y-axis = proportion)
plt.figure(figsize=(10, 6))
start_dates = consistent_sorted_podcasts['min'].dt.year
# start_dates[0:10]
sns.ecdfplot(x=start_dates)
plt.savefig("year_data_ecdf_proportion.png")
plt.close()

In [47]:
# ECDF plot (y-axis = count)
plt.figure(figsize=(10, 6))
start_dates = consistent_sorted_podcasts['min'].dt.year
# start_dates[0:10]
sns.ecdfplot(x=start_dates, stat='count')
plt.savefig("year_data_ecdf_proportion_count.png")
plt.close()

In [59]:
#------------------- Plot ------------------------ 
# Plot 1,2,3: the most popular languages, categories, and explicit statuses during each presidency
# Plot 4: most popular categories of all time


# Get the unique titles in consistent_sorted_podcasts
consistent_titles = consistent_sorted_podcasts['podcast title']

# Filter df to only include rows with these titles
df_clean = Df[Df['podcast title'].isin(consistent_titles)]


#--------------------------------------------------------------------------

# Plot 1: the 30 most popular podcast categories during each US presidency

# gov_periods = {
#     'Bill Clinton': ('1993-01-20', '2001-01-20'),
#     'George W. Bush': ('2001-01-20', '2009-01-20'),
#     'Barack Obama': ('2009-01-20', '2017-01-20'),
#     'Donald Trump': ('2017-01-20', '2021-01-20'),
#     'Joe Biden': ('2021-01-20', 'today')
# }
# # Analyze each government period
# for name, (start, end) in gov_periods.items():
#     # Filter the data for the current government period
#     mask = (df_clean['publish date'] >= start) & (df_clean['publish date'] < end)
#     subset = df_clean[mask]

#     # Count the number of podcasts in each category
#     category_counts = subset['category'].value_counts()

#     # Get the 30 most popular categories
#     top_categories = category_counts[:30]

#     # Check if top_categories is empty
#     if not top_categories.empty:
#         # Create a new figure
#         plt.figure(figsize=(40, 10))

#         # Create a bar plot using seaborn
#         sns.barplot(x=top_categories.values, y=top_categories.index)

#         # Add labels and title
#         plt.title(f'Top 30 Podcast Categories during {name}\'s Presidency')
#         plt.xlabel('Number of Podcasts')
#         plt.ylabel('Category')

#         filename = f"{name.replace(' ', '_')}_popular_categories.png"
#         plt.savefig(filename)

#         plt.close()
#     else:
#         print(f"No podcasts found during {name}'s presidency.")  

#--------------------------------------------------------------------------

# Plot 2: the 30 most popular podcast languages during each US presidency
# gov_periods = {
#     'Bill Clinton': ('1993-01-20', '2001-01-20'),
#     'George W. Bush': ('2001-01-20', '2009-01-20'),
#     'Barack Obama': ('2009-01-20', '2017-01-20'),
#     'Donald Trump': ('2017-01-20', '2021-01-20'),
#     'Joe Biden': ('2021-01-20', 'today')
# }
# # Analyze each government period
# for name, (start, end) in gov_periods.items():
#     # Filter the data for the current government period
#     mask = (df_clean['publish date'] >= start) & (df_clean['publish date'] < end)
#     subset = df_clean[mask]

#     # Count the number of podcasts in each category
#     language_counts = subset['language'].value_counts()

#     # Get the 30 most popular categories
#     top_languages = language_counts[:30]

    # Check if top_categories is empty
#     if not top_languages.empty:
#         # Create a new figure
#         plt.figure(figsize=(40, 10))

#         # Create a bar plot using seaborn
#         sns.barplot(x=top_languages.values, y=top_languages.index)

#         # Add labels and title
#         plt.title(f'Top 30 Podcast Languages during {name}\'s Presidency')
#         plt.xlabel('Number of Podcasts')
#         plt.ylabel('Language')

#         filename = f"{name.replace(' ', '_')}_popular_languages.png"
#         plt.savefig(filename)

#         plt.close()
#     else:
#         print(f"No podcasts found during {name}'s presidency.")  
  

## --------------------------------------------------------------------------

# Plot 3: the populaity of explicit podcast during each presidency

# Won't set X-axis to the same metric because we won't be able to observe anything in Bush and Clinton's presidency by doing this

# However, it's a good starting point. I've commented out the code to set X-axis's limit to the highest frequency, come back if needed

# gov_periods = {
#     'Bill Clinton': ('1993-01-20', '2001-01-20'),
#     'George W. Bush': ('2001-01-20', '2009-01-20'),
#     'Barack Obama': ('2009-01-20', '2017-01-20'),
#     'Donald Trump': ('2017-01-20', '2021-01-20'),
#     'Joe Biden': ('2021-01-20', 'today')
# }

# --------- Required part to set X-axis upper limit to the highest freq
# highest_freq = 0
# # Get the highest frequency and use it as the upper limit of the X-axis
# for name, (start, end) in gov_periods.items():
#     # Filter the data for the current government period
#     mask = (df_clean['publish date'] >= start) & (df_clean['publish date'] < end)
#     subset = df_clean[mask]

#     # Count the number of explicit and non-explicit podcasts
#     explicit_counts = subset['explicit'].value_counts()
#     highest_freq = max(highest_freq, explicit_counts.iloc[0])
# --------- Required part to set X-axis upper limit to the highest freq    

# Analyze each government period
# for name, (start, end) in gov_periods.items():
#     # Filter the data for the current government period
#     mask = (df_clean['publish date'] >= start) & (df_clean['publish date'] < end)
#     subset = df_clean[mask]

#     # Count the number of explicit and non-explicit podcasts
#     explicit_counts = subset['explicit'].value_counts()

#     # Plot the counts
#     plt.figure(figsize=(10, 6))
#     explicit_counts.plot(kind='barh', color=['blue', 'red'])
#     plt.title(f'Explicitness Counts during {name}\'s Presidency')
#     plt.xlabel('Number of Podcasts')
#     plt.ylabel('Explicitness')

    
# # --------- Required part to set X-axis upper limit to the highest freq
#     # Set the x-axis limits
#     plt.xlim(0, highest_freq)
    
#     # Set the xticks
#     xticks = np.arange(0, highest_freq, 0.05 * 1e7)  # adjust the step size as needed
#     plt.xticks(xticks)
# # --------- Required part to set X-axis upper limit to the highest freq


#     filename = f"{name.replace(' ', '_')}_explicit_counts.png"
#     plt.savefig(filename)

#     plt.close()
    
    
#--------------------------------------------------------------------------

# Plot 4: the most popular podcasts category of all time:

    # Count the number of explicit and non-explicit podcasts
# cat_counts = df_clean['category'].value_counts()
# top_cat = cat_counts[:30]
# # Plot the counts
# plt.figure(figsize=(40, 10))

# sns.barplot(x=top_cat.values, y=top_cat.index)
# plt.title(f'30 Most common podcast topics since 1992-')
# plt.xlabel('Number of Podcasts')
# plt.ylabel('Topic')

# filename = "popular_categories_of_all_time.png"
# plt.savefig(filename)

# plt.close()


#--------------------------------------------------------------------------
# # Plot 5: time series plot that plots the number of podcasts by each category from 1990 - 2000
# # Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 4 categories (only has 4 categories during that time)
# top_categories = df_clean_copy['category'].value_counts().index[:5]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend by Year for Top 4 Categories from 1990 to 2000')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(1990, 2000)
# plt.yticks(np.arange(0, 200, 10))
# plt.ylim(0, 200)

# # Save the figure
# plt.savefig('1990_2000_series_plot.png')

# # Close the figure
# plt.close()

#-------------------------------------------------------------------

# Plot 6: time series plot that plots the number of podcasts by each category from 2000 - 2010
# Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 10 categories
# top_categories = df_clean_copy['category'].value_counts().index[:10]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend by Year for Top 10 Categories from 2000 to 2010')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(2000, 2010)
# plt.yticks(np.arange(0, 7000, 200))
# plt.ylim(0, 7000)

# # Save the figure
# plt.savefig('2000_2010_series_plot.png')

# # Close the figure
# plt.close()

# # -------------------------------------------------------------------

# # Plot 7: time series plot that plots the number of podcasts by each category from 2010 - 2023
# # Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 10 categories
# top_categories = df_clean_copy['category'].value_counts().index[:10]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend for Top 10 Categories from 2010 to 2023')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(2010, 2023)
# plt.yticks(np.arange(0, 500000, 20000))
# plt.xticks(np.arange(2010, 2023, 1))
# plt.ylim(0, 500000)

# # Save the figure
# plt.savefig('2010_2023_series_plot.png')

# # Close the figure
# plt.close()

# -------------------------------------------------------------------

# Plot 8: time series plot that plots the number of podcasts by each category from 1990 - 2023
# Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 10 categories
# top_categories = df_clean_copy['category'].value_counts().index[:10]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend for Top 10 Categories from 1990 to 2023')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(1990, 2023)
# plt.yticks(np.arange(0, 500000, 10000))
# plt.ylim(0, 500000)

# # Save the figure
# plt.savefig('1990_2023_series_plot.png')

# # Close the figure
# plt.close()




In [397]:
# # Plot 5: time series plot that plots the number of podcasts by each category from 1990 - 2000
# # Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 4 categories (only has 4 categories during that time)
# top_categories = df_clean_copy['category'].value_counts().index[:5]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend by Year for Top 4 Categories from 1990 to 2000')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(1990, 2000)
# plt.yticks(np.arange(0, 200, 10))
# plt.ylim(0, 200)

# # Save the figure
# plt.savefig('1990_2000_series_plot.png')

# # Close the figure
# plt.close()

#-------------------------------------------------------------------

# Plot 6: time series plot that plots the number of podcasts by each category from 2000 - 2010
# Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 10 categories
# top_categories = df_clean_copy['category'].value_counts().index[:10]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend by Year for Top 10 Categories from 2000 to 2010')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(2000, 2010)
# plt.yticks(np.arange(0, 7000, 200))
# plt.ylim(0, 7000)

# # Save the figure
# plt.savefig('2000_2010_series_plot.png')

# # Close the figure
# plt.close()

# # -------------------------------------------------------------------

# # Plot 7: time series plot that plots the number of podcasts by each category from 2010 - 2023
# # Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 10 categories
# top_categories = df_clean_copy['category'].value_counts().index[:10]
# df_top_categories = df_clean_copy[df_clean_copy['category'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'category']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='category', data=df_counts)

# # Add labels and title
# plt.title('Podcasts Trend for Top 10 Categories from 2010 to 2023')
# plt.xlabel('Year')
# plt.ylabel('Podcasts Count')
# plt.xlim(2010, 2023)
# plt.yticks(np.arange(0, 500000, 20000))
# plt.xticks(np.arange(2010, 2023, 1))
# plt.ylim(0, 500000)

# # Save the figure
# plt.savefig('2010_2023_series_plot.png')

# # Close the figure
# plt.close()

# # -------------------------------------------------------------------

# # Plot 8: time series plot that plots the number of podcasts by languages from 1990 - 2023
# # Convert 'publish date' to datetime and extract the year
df_clean_copy = df_clean.copy()
df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# Filter the top 15 languages
top_categories = df_clean_copy['language'].value_counts().index[:15]
df_top_categories = df_clean_copy[df_clean_copy['language'].isin(top_categories)]

# Count the number of podcasts per category per year
df_counts = df_top_categories.groupby(['year', 'language']).size().reset_index(name='count')

# Create a new figure
plt.figure(figsize=(25, 10))

# Create a line plot using seaborn
sns.lineplot(x='year', y='count', hue='language', data=df_counts)

# Add labels and title
plt.title('Top 10 Languages from 1990 to 2023')
plt.xlabel('Year')
plt.ylabel('Podcast Counts')
plt.xlim(1990, 2023)
plt.yticks(np.arange(0, 3000000, 100000))
plt.ylim(0, 3000000)

# Save the figure
plt.savefig('1990_2023_lang_series_plot.png')

# Close the figure
plt.close()

# -------------------------------------------------------------------

# Plot 9: time series plot that plots the number of podcasts by explicit status from 1990 - 2023
# Convert 'publish date' to datetime and extract the year
# df_clean_copy = df_clean.copy()
# df_clean_copy.loc[:, 'year'] = pd.to_datetime(df_clean_copy['publish date']).dt.year

# # Filter the top 2 explicit statuses
# top_categories = df_clean_copy['explicit'].value_counts().index[:2]
# df_top_categories = df_clean_copy[df_clean_copy['explicit'].isin(top_categories)]

# # Count the number of podcasts per category per year
# df_counts = df_top_categories.groupby(['year', 'explicit']).size().reset_index(name='count')

# # Create a new figure
# plt.figure(figsize=(25, 10))

# # Create a line plot using seaborn
# sns.lineplot(x='year', y='count', hue='explicit', data=df_counts)

# # Add labels and title
# plt.title('Explicit Podcasts from 1990 to 2023')
# plt.xlabel('Year')
# plt.ylabel('Podcast Counts')
# plt.xlim(1990, 2023)
# plt.yticks(np.arange(0, 5000000, 100000))
# plt.ylim(0, 5000000)

# # Save the figure
# plt.savefig('1990_2023_explicit_series_plot.png')

# # Close the figure
# plt.close()

In [60]:
df_clean.columns

Index(['podcast title', 'publish date', 'category', 'language', 'explicit'], dtype='object')

In [70]:
df_clean.to_csv("df_clean.csv")

In [69]:
df_no_nat = df_clean.dropna(subset=['category'])

catsList = df_no_nat["category"].str.split(',')

# Initialize a dictionary to store the counts of each pair of categories
catsDict = defaultdict(int)

def extractPairs(item): 
    for i in range(len(item)): 
        for j in range(i+1, len(item)): 
            # Sort the pair so that 'A, B' and 'B, A' are considered the same
            pair = tuple(sorted([item[i], item[j]]))
            catsDict[pair] += 1

# Extract all pairs from the categories list
for item in catsList: 
    extractPairs(item)

df_pairs = pd.DataFrame.from_dict(catsDict, orient='index', columns=['count']).reset_index()

# Create two separate columns for each element of the pair
df_pairs[['Category1', 'Category2']] = pd.DataFrame(df_pairs['index'].tolist(), index=df_pairs.index)

# Drop the original 'index' column
df_pairs = df_pairs.drop(columns=['index'])

# Create a pivot table for the heatmap
pivot_table = df_pairs.pivot(index='Category1', columns='Category2', values='count').fillna(0)

# Create a new figure
plt.figure(figsize=(12, 12))

# Create a heatmap using seaborn
sns.heatmap(pivot_table, cmap='YlGnBu')

# Add labels and title
plt.title('Heatmap of Podcast Category Pairs')
plt.xlabel('Category')
plt.ylabel('Category')

# Save the figure to a file and do not display it
plt.savefig('heatmap.png')
plt.close()

NameError: name 'defaultdict' is not defined

In [63]:
# #boilerplate code for subplots 
# fig, ax = plt.subplots(2, 2, figsize=(12,12))

# #replace this with whatever your end up plotting
# #ax[0][0] just says which subplot to plot onto 
# #ax[0][0].scatter([1,2,3],[1,2,3])

# #or with seaborn use 
# #sns.heatmap(pivot_table, ax=ax[0][0])

# ax[0][0]

In [ ]:
# df_pairs.value_counts

In [421]:
# Top 20 correlated keywords (Haven't figured out)
# catList = df_clean["category"].str.split(',')

# # Initialize a dictionary to store the counts of each pair of categories
# catDict = defaultdict(int)

# def extractPairs(item): 
#     for i in range(len(item)): 
#         for j in range(i+1, len(item)): 
#             # Sort the pair so that 'A, B' and 'B, A' are considered the same
#             pair = tuple(sorted([item[i], item[j]]))
#             catsDict[pair] += 1

# # Extract all pairs from the categories list
# for item in catList: 
#     if isinstance(item, list):  # Add this line to check if item is a list before calling extractPairs
#         extractPairs(item)

# # Convert the dictionary to a DataFrame and reset index
# df_pair = pd.DataFrame.from_dict(catDict, orient='index', columns=['count']).reset_index()

# # Create two separate columns for each element of the pair
# df_pair['Category1'] = df_pair['index'].apply(lambda x: x[0])
# df_pair['Category2'] = df_pair['index'].apply(lambda x: x[1])

# # Drop the original 'index' column
# df_pair = df_pair.drop(columns=['index'])

# # Create a pivot table for the heatmap
# pivot_table = df_pair.pivot(index='Category1', columns='Category2', values='count').fillna(0)

# # Calculate correlation matrix
# corr_matrix = pivot_table.corr()

# # Unstack correlation matrix and sort by correlation coefficient
# corr_pairs = corr_matrix.unstack().sort_values(ascending=False)

# # Get top 20 most correlated pairs (excluding self-correlations)
# top_corr_pairs = corr_pairs[corr_pairs != 1.0][:20]

# # Create a new DataFrame for these pairs and their correlations
# df_top_corr = pd.DataFrame(top_corr_pairs.index.tolist(), columns=['level_0', 'level_1'])
# df_top_corr['correlation'] = top_corr_pairs.values

# # Pivot this DataFrame to create a table suitable for a heatmap
# pivot_table_top_corr = df_top_corr.pivot(index='level_0', columns='level_1', values='correlation')

# # Create a new figure
# plt.figure(figsize=(10, 10))

# # Create a heatmap using seaborn
# sns.heatmap(pivot_table_top_corr, cmap='YlGnBu')

# # Add labels and title
# plt.title('Heatmap of Top 20 Most Correlated Podcast Category Pairs')
# plt.xlabel('Category')
# plt.ylabel('Category')

# # Save the figure to a file and do not display it
# plt.savefig('heatmap.png')
# plt.close()

In [43]:
# # Plot an empirical-cdf graph regarding category
# df_no_nat_cat = df_clean.dropna(subset=['category'])
# top_categories = df_no_nat_cat['category'].value_counts().index[:10]

# # Create a new figure
# plt.figure(figsize=(14, 10))

# # Calculate and plot the ECDF for each of the top categories
# for category in top_categories:
#     data = df_no_nat_cat[df_no_nat_cat['category'] == category]['category']  
#     x = np.sort(data)
#     y = np.arange(1, len(x)+1) / len(x)
#     sns.lineplot(x=x, y=y, label=category)

# # Add labels and title
# plt.title('Empirical Cumulative Distribution Function (ECDF) by Category')
# plt.xlabel('Value')
# plt.ylabel('Cumulative Probability')

# # Add a legend
# plt.legend(title='Category')

# # rotate x axis so that labels won't overlap 
# plt.xticks(rotation=45)

# # Save the figure to a file and do not display it
# plt.savefig('emp_cdf_cat.png')

# # Close the figure to prevent it from displaying in your notebook or script output
# plt.close()


In [44]:
# # ECDF plot, the dates of the first episode on the x axis
# data = pd.to_datetime(df_clean['publish date']).dt.year

# # Calculate the ECDF
# x = np.sort(data)
# y = np.arange(1, len(x)+1) / len(x)

# # Create a new figure
# fig, ax = plt.subplots(figsize=(10, 8))

# # Create a line plot for the ECDF using seaborn
# sns.lineplot(x=x, y=y, ax=ax)

# # Limit the number of x-labels to 30
# ax.xaxis.set_major_locator(MaxNLocator(nbins=30))

# # Add labels and title
# plt.title('Empirical Cumulative Distribution Function (ECDF) of Podcast Age')
# plt.xlabel('Year of First Episode')
# plt.ylabel('Cumulative Proportion')

# plt.xticks(rotation=45)
# # Show the plot
# plt.savefig("ecdf_first_episode.png")
# plt.close()

In [440]:
# df_clean['podcast title'][0:5]
# df_clean.columns

In [256]:
# print(consistent_sorted_podcasts[consistent_sorted_podcasts['min'] == '2007-07-06 15:30:06'])
# print(consistent_sorted_podcasts[consistent_sorted_podcasts['median'] == '2017-07-31 21:59:35.00']['podcast title'].to_string(index=False))

Sound Design Live - Career building interviews on live sound, theatre, AV, recording, and sound system tuning


In [ ]:
# Please don't run the code below:

In [ ]:
# sort podcasts by time span
# sorted_podcasts_by_time_span = grouped.sort_values('time span', ascending=False)

In [ ]:
# sorted_podcasts_by_time_span.to_csv('sorted_podcasts.csv')